# Proyecto Final Análisis Aplicado
Jacqueline Lira Chávez **167334**  

El objetivo de este proyecto es implementar los siguientes algoritmos
* BFGS
* Búsqueda Lineal de Newton con Modificación a la Hessiana
* Algoritmo de Newton
* Búsqueda Lineal Base (sin modificación a la Hessiana)

## Código

#### Imports

In [32]:
import numpy as np
from random import random
from IPython.display import display, Math

#### Gradiente
Código que calcula el gradiente de la función f en el punto xk utilizando la derivada mediante la diferencia central.
$$f'(x_k) = \frac{f(x_k+k)-f(x_k-k)}{2k}$$

In [33]:
def gradiente(f, xk, eps = 1e-6):
    size = len(xk)
    grad = np.zeros(size, dtype = float)
    for i in range(size):
        x_f = xk.copy(); 
        x_f[i] += eps
        x_b = xk.copy(); x_b[i] -= eps
        grad[i] = (f(x_f) - f(x_b)) / (2 * eps)
    return grad

#### Hessiana
Código que calcula el hessiana de la función f en el punto $x_k$ utilizando la derivada mediante la diferencia 
central, utilizando la función gradiente. $$f''(x_k) = \frac{f'(x_k+h)-f'(x_k-h)}{2h}$$

In [34]:
def hessiana(f,xk, eps = 1e-3):
    size = len(xk)
    hess = np.zeros((size, size))
    for j in range(size):
        # Primera derivada
        dx_ff = xk.copy(); dx_ff[j] += eps;
        dx_bb = xk.copy(); dx_ff[j] += eps;
        grad_f = gradiente(f, dx_ff)
        grad_b = gradiente(f, dx_bb)
        for i in range(j+1):
            # Segunda derivada
            hess[i, j] = (grad_f[i] - grad_b[i]) / (2 * eps)
            hess[j, i] = (grad_f[i] - grad_b[i]) / (2 * eps)
    return hess

#### Newton
Este es el algoritmo de Newton básico, en dónde $\alpha$ le es dada. En este no incluimos la modificación a la Hessiana

In [190]:
def Newton(f, xk, alpha, eps = 1e-6, max_iter = 1000): 
    for k in range(max_iter):
        grad = gradiente(f, xk)
        if(np.linalg.norm(grad) < eps): break
        hess = hessiana(f, xk)
        try:
            pk = np.linalg.solve(hess, -grad)
        except np.linalg.LinAlgError as err:
            print("La hessiana no es positiva definida")
            break
        xk = xk + alpha * pk
    return xk

#### Backtracking Line Search (Algoritmo 3.1)
Algoritmo que determina la cantidad a moverse dentro de una dirección de búsqueda.
Notamos que para métodos Newton y quasi-Newton se útiliza el valor inicial $\alpha_0 = 1$

In [40]:
def BLS(f, xk, alpha_0, pk, c):
    alpha = alpha_0
    while f(xk + alpha * pk) > f(xk) + c * alpha * gradiente(f, xk) @ pk:
        p = random()
        alpha = p * alpha
    return alpha

#### Búsqueda Lineal de Newton
En este método se implementa la búsqueda lineal de Newton, con un $\alpha$ cambiante, pero todavía no incluímos la modificación a la hessiana.

In [207]:
def LSN(f, xk):
    for k in range(100):
        Bk = hessiana(f, xk)
        try:
            pk = np.linalg.solve(Bk, - gradiente(f, xk))
        except np.linalg.LinAlgError as err:
            print("La hessiana no es positiva definida")
            break
        xk = xk + BLS(f, xk, 1, pk, 0.5) * pk
    return xk

#### Cholesky with Added Múltiple of the Identity (Algoritmo 3.3)
Buscamos $\tau$ de tal forma que la matriz $A + \tau I$ es positiva semidefinida, por lo que iterativamente se va aumentando $\tau$ hasta que se cumpla

In [7]:
def CAMI(A):
    beta = 0.001
    n = len(A)
    if min(np.diag(A)) > 0:
        t = 0
    else:
        t = -min(np.diag(A)) + beta
    for k in range(100):
        try:
            np.linalg.cholesky(A + t * np.identity(n))
        except np.linalg.LinAlgError as err:
            t = max(2 * t, beta)
        else:
            break
    return A + t * np.identity(n)

#### Line Search Newton with Modification (Algoritmo 3.2)
Se le hace una modificación al algoritmo de Newton para que se puede utilizar aunque la matriz hessiana no sea semipositiva definida.

In [8]:
def LSNM(f, xk):
    for k in range(100):
        Bk = hessiana(f, xk)
        try:
            np.linalg.cholesky(Bk)
        except np.linalg.LinAlgError as err:
            Bk = CAMI(Bk)   
        pk = np.linalg.solve(Bk, - gradiente(f, xk))
        xk = xk + BLS(f, xk, 1, pk, 0.5) * pk
    return xk

#### BFGS
Algoritmo nombrado por sus descubridores, Broyden, Fletcher, Goldfarb y Shanno.

In [73]:
def BFGS(f, xk, Hk, eps = 1e-6, max_iter = 10000):
    I = np.identity(len(xk))
    for k in range(max_iter):
        grad = gradiente(f, xk)
        if(np.linalg.norm(grad) < eps): break
        #Dirección de Descenso
        pk = -Hk @ grad
        #Calculamos el tamaño del paso
        alpha = BLS(f, xk, 1, pk, 0.5)
        #Nueva x
        sk = alpha*pk; sk_t = sk.transpose()
        xk = xk + sk
        #Calculamos yk y phi_k
        yk = gradiente(f,xk) - grad; yk_t = yk.transpose();
        if(yk_t@sk == 0): break
        phi_k = 1.0/(yk_t@sk)
        #Calculamos la nueva hessiana inversa
        Hk = (I - phi_k*sk@yk_t)@Hk@(I - phi_k*sk@yk_t) + phi_k * sk @ sk_t
    return xk

## Pruebas
Para probar nuestro algoritmo utilizamos la función de Rosenbrock definida como $f(x,y)=(a-x)^2+b(y-x^2)^2$, que tiene un mínimo local en $(a,a^2)$, con diversos valores para $a$ y $b$ y puntos iniciales $(x,y)$

#### Función de Rosenbrock

In [43]:
def Rosenbrock(x, a = 1, b = 100):
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

##### Prueba 1
$(a,b) = (1, 100)$

$x = (0.9, 1.1)$

Primero probamos con un valor cercano al verdadero.

###### Prueba Newton

In [81]:
x = np.array([0.9, 1.1])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = Newton(Rosenbrock, x, 0.1)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###### Prueba LSN

In [206]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

x = np.array([0.9, 1.1])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSN(Rosenbrock, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###### Prueba LSNM

In [83]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

x = np.array([0.9, 1.1])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSNM(Rosenbrock, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###### Prueba BFGS

In [84]:
x = np.array([0.9, 1.1])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = BFGS(Rosenbrock, x, np.identity(len(x)))
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

##### Prueba 2
$(a,b) = (1, 100)$

$(x,y) = (8, -4)$  

Utilizamos un vector más alejado del óptimo, en dónde también podemos ver que funcione correctamente el algoritmo de Cholesky con Identidad Múltiple Agregada

###### Prueba Newton

In [86]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = Newton(Rosenbrock, x,0.1)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###### Prueba LSN

In [205]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSN(Rosenbrock, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###### Prueba LSNM

In [88]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSNM(Rosenbrock, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###### Prueba BFGS 

In [89]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = BFGS(Rosenbrock, x, np.identity(len(x)))
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

##### Prueba 3
$(a,b) = (2, 5)$

$(x,y) = (8, -4)$  

Cambiamos el valor de $a$ y $b$

In [100]:
def Rosenbrock_2(x, a = 2, b = 5):
    return (a - x[0])**2 + b * (x[1] - x[0]**2)**2

###### Prueba Newton

In [101]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock_2(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = Newton(Rosenbrock_2, x,0.1)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock_2(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###### Prueba LSN

In [94]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock_2(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSN(Rosenbrock_2, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock_2(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###### Prueba LSNM

In [95]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock_2(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = LSNM(Rosenbrock_2, x)
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock_2(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

##### Prueba BFGS

In [97]:
x = np.array([8, -4])
display(Math('x: [{},{}]'.format(x[0],x[1])))
f_x = Rosenbrock_2(x)
display(Math('f(x): {}'.format(f_x)))
x_aprox = BFGS(Rosenbrock_2, x, np.identity(len(x)))
display(Math('x_0: [{},{}]'.format(x_aprox[0],x_aprox[1])))
f_x_aprox = Rosenbrock_2(x_aprox)
display(Math('f(x_0): {}'.format(f_x_aprox)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Optimización de función de distancia

In [111]:
import math
import pandas as pd

##### Procesamiento de Datos
Filtramos los datos por fecha y nos quedamos con solo las variables relevantes

In [143]:
data=pd.read_csv('crime_data.csv')
#Filtramos por fecha
fecha_inicio = '2019-12-25'
fecha_fin = '2019-12-25'
data = data.loc[((data['date']>=fecha_inicio) & (data['date']<=fecha_fin))]
#Nos quedamos con sólo longitud y latitud
data = data[{'lat','long'}]
#Lo convertimos en un numpy array
crimes = data.to_numpy()

Creamos un vector de $n$ camaras, en dónde inicialmente estas se encuentran dónde ocurrieron los primeros $n$ crimenes. 

In [144]:
n = 8 # Número de cameras
cameras = crimes[:8].copy()

#### Haversine Formula
Utilizamos la fórmula de haversine para medir la distancia entre dos puntos geográficos (latitud, longitud)

In [181]:
# Regresa la distancia entre dos ountos que se encuentran  en lat y lon
def haversine(p1, p2):
    # Pasamos de decimales a radianes
    p1_lat = math.radians(p1[0]);p1_lon = math.radians(p1[1]);
    p2_lat = math.radians(p2[0]);p2_lon = math.radians(p2[1]);
    # formula de haversine
    h = math.sin((p2_lat-p1_lat)/2)**2 + math.cos(p1_lat) * math.cos(p2_lat) * math.sin((p2_lon-p1_lon)/2)**2
    c = 2 * math.asin(math.sqrt(h)) 
    r = 6371 # Radio de la tierra en km
    return c * r

In [182]:
def distancia_min(x, crimenes = crimes):
    #Suponemos x una función con latitud y longitud
    dist = 0
    for i in range(len(x)):
        for j in range(len(crimenes)):
            dist += haversine(x[i], crimenes[j])
        for j in range(len(x)):
            if(i != j):
                dist += 1/(haversine(x[i], x[j]))
    return dist 

In [187]:
distancia_min(cameras)

3637.3671451270766